<a href="https://colab.research.google.com/github/SandipanMajhi/Document-Analysis-/blob/main/Mnist_feature_engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.datasets as datasets
import numpy as np
import cv2
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
train_set = datasets.MNIST(root = "./data", train=True, download=True, transform=None)
test_set = datasets.MNIST(root = "./data", train=False, download = True, transform=None)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [ ]:
train_arrays = []
train_labels = []
test_arrays = []
test_labels = []

def pil_to_np(stock_set):
  target_set = []
  target_labels = []
  for i in range(0, len(stock_set)):
    target_set.append(np.array(stock_set[i][0]))
    target_labels.append(np.array(stock_set[i][1]))

  return target_set, target_labels

train_arrays, train_labels = pil_to_np(train_set)
test_arrays, test_labels = pil_to_np(test_set)

In [ ]:
### Creating contour maps from grayscale image

def gray_to_contours(dataset):
  contours = []
  for i in range(0, len(dataset)):
    contour, hierarchy = cv2.findContours(image=dataset[i], mode=cv2.RETR_TREE, method=cv2.CHAIN_APPROX_NONE)
    contour_transform = np.zeros((dataset[i].shape))
    contour_transform = cv2.drawContours(image = contour_transform, contours = contour , contourIdx=-1, color=(255))
    contours.append(contour_transform)
  
  return contours

train_contours = gray_to_contours(train_arrays)
test_contours = gray_to_contours(test_arrays)

In [ ]:
class Map_Data():
  def __init__(self, dataset):
    self.bdata = []
    self.bcurves = []
    self.dataset= dataset

    ### For multiple curves
    self.bcurves1 = [] 

  def create_maps(self):
    for i in range(len(self.dataset)):
      img = self.dataset[i]
      img[img == 255] = 1

      ### Boundary maps
      temp_maps = [] 
      temp_maps.append(np.array(left_row_maps(img)))
      temp_maps.append(np.array(right_row_maps(img)))
      temp_maps.append(np.array(top_col_maps(img)))
      temp_maps.append(np.array(bottom_col_maps(img)))

      ### Curve maps
      temp_curves = []
      lcurve = get_curvatures(left_row_maps(img))
      rcurve = get_curvatures(right_row_maps(img))
      tcurve = get_curvatures(top_col_maps(img))
      bcurve = get_curvatures(bottom_col_maps(img))
      temp_curves.append(lcurve)
      temp_curves.append(rcurve)
      temp_curves.append(tcurve)
      temp_curves.append(bcurve)

      ### Save the datasets
      self.bdata.append(temp_maps)
      self.bcurves.append(temp_curves)

  
  def left_row_maps(img):
    maps = []
    for i in range(0, img.shape[0]):
      dist = 0
      for j in range(0, img.shape[0]):
        dist = dist + 1 
        if img[i][j] == 1:
          dist = dist + 1
          break
      maps.append(dist)
    return maps 

  def right_row_maps(img):
    maps = []
    for i in range(0, img.shape[0]):
      dist  = 0
      for j in range(img.shape[0]-1, -1, -1):
        dist = dist + 1
        if img[i][j] == 1:
          dist = dist  + 1
          break
      maps.append(dist)
    return maps

  def top_col_maps(img):
    maps = []
    for i in range(0, img.shape[0]):
      dist = 0
      for j in range(0, img.shape[0]):
        dist = dist + 1 
        if img[j][i] == 1:
          dist = dist + 1 
          break
      maps.append(dist)
    return maps

  def bottom_col_maps(img):
    maps = []
    for i in range(img.shape[0] - 1, -1,-1):
      dist = 0
      for j in range(0, img.shape[0]):
        dist = dist + 1 
        if img[j][i] == 1:
          dist = dist + 1 
          break
      maps.append(dist)
    return maps

  def get_curvatures(maps, step_size = 1):
    curves = []
    for i in range(1, img.shape[0], step_size):
      grad = (maps[i]-maps[i-1])
      curves.append(grad)
    return curves



In [ ]:
train_maps = Map_Data(train_contours)
test_maps = Map_Data(test_contours)

train_maps.create_maps()
test_maps.create_maps()

In [ ]:
### Testing



[array([28, 28, 28, 28, 28, 14, 10,  9,  9, 10, 11, 13, 13, 14, 15, 16, 17,
        19, 16, 14, 12, 10,  8,  6,  6, 28, 28, 28]),
 array([28, 28, 28, 28, 28,  6,  6,  7, 12, 12, 16, 15, 15, 12, 11, 10, 10,
         9,  9, 10, 11, 12, 14, 16, 18, 28, 28, 28]),
 array([28, 28, 28, 28, 25, 25, 24,  9,  8,  8,  8,  8,  7,  7,  7,  7,  7,
         7,  7,  7,  7,  7,  7,  7, 28, 28, 28, 28]),
 array([28, 28, 28, 28,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  8,
         8,  8,  8,  9, 24, 25, 25, 28, 28, 28, 28])]